In [1]:
import os

In [2]:
%pwd

'e:\\Workspace\\End-to-End-Mlops\\research'

In [3]:
os.chdir("E:/Workspace/End-to-End-Mlops")

In [4]:
%pwd

'E:\\Workspace\\End-to-End-Mlops'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Data_Evaluation_Config:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_name: Path
    all_params: dict
    target_columns: str

In [13]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories
from src.mlProject.utils.common import read_yaml,create_directories, save_json_data


In [31]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH,para_file_path=PARAMS_FILE_PATH):

        
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(para_file_path)

        create_directories([self.config.artifacts_root])

    def get_risk_evaluation_config(self)->Data_Evaluation_Config:

        create_directories([self.config.model_evaluation.root_dir])

        return Data_Evaluation_Config(root_dir=self.config.model_evaluation.root_dir,
                                      test_data_path=self.config.model_evaluation.test_data_path,
                                      model_path=self.config.model_evaluation.model_path,
                                      metrics_file_name=self.config.model_evaluation.metrics_file_name,
                                      all_params=self.params.XGB_MODEL,
                                      target_columns=self.schema.TARGET_COLUMN.name
                                      )


In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from urllib.parse import urlparse
import joblib


In [35]:
class Risk_Evaluation:
    def __init__(self,config:ConfigurationManager):
        self.config = config


    def evaluate_model(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return mae, rmse, r2
    
    def save_result(self):

        
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop(self.config.target_columns,axis=1)
        test_y=test_data[self.config.target_columns]

        pred=model.predict(test_x)
        mae,rmse,r2=self.evaluate_model(test_y, pred)

        # Saving metrics as local

        scores={"rmse":rmse,"mae":mae, "r2":r2}

        save_json_data(path=Path(self.config.metrics_file_name), data=scores)


In [36]:
try:
    cm = ConfigurationManager()
    cm_config = cm.get_risk_evaluation_config()
    re = Risk_Evaluation(config=cm_config)
    re.save_result()
except Exception as e:
    raise e

--- Logging error ---
Traceback (most recent call last):
  File "e:\ML_Project\ml_project\lib\logging\__init__.py", line 1082, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Call stack:
  File "e:\ML_Project\ml_project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\ML_Project\ml_project\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", lin

checking1
check2


e:\ML_Project\ml_project\lib\site-packages\xgboost\core.py:158: UserWarning: [23:57:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\data\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
--- Logging error ---
Traceback (most recent call last):
  File "e:\ML_Project\ml_project\lib\logging\__init__.py", line 1082, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Call stack:
  File "e:\ML_Project\ml_project\lib\runpy.py", line 197, in _run_module_